In [2]:
# Importar bibliotecas
import pandas as pd
import numpy as np
from bcb import sgs
# Dados do IDP/BP - acum. 12m - US$ (milhões) (SGS/BCB)
dados_sgs = sgs.get(codes = {"valor": 24422}, start = "2021-01-01", end = "2024-11-30")
# Dados de expectativas do IPCA (Focus/BCB)
dados_focus = pd.read_csv(
  filepath_or_buffer = "https://olinda.bcb.gov.br/olinda/servico/Expectativas/versao/v1/odata/ExpectativasMercadoAnuais?%24filter=%28Indicador%20eq%20%27IPCA%27%20or%20Indicador%20eq%20%27IGP-M%27%29%20and%20Data%20ge%20%272023-01-01%27%20and%20Data%20le%20%272023-02-12%27&%24format=text/csv&%24orderby=Data%20desc",
  decimal = ","
  )

In [3]:
dados_focus

,Indicador,IndicadorDetalhe,Data,DataReferencia,Media,Mediana,DesvioPadrao,Minimo,Maximo,numeroRespondentes,baseCalculo
0,IGP-M,NaN,2023-02-10,2023,4.5840,4.5955,0.8950,2.6607,7.8000,73,0
1,IGP-M,NaN,2023-02-10,2024,4.3685,4.0650,1.1936,2.8000,9.9000,56,0
2,IGP-M,NaN,2023-02-10,2025,4.0136,3.9850,0.7051,3.0000,6.0000,50,0
3,IGP-M,NaN,2023-02-10,2026,3.8977,3.6000,0.6957,2.7900,5.5000,47,0
4,IGP-M,NaN,2023-02-10,2027,3.8798,3.8700,0.7446,2.5800,5.5000,42,0
...,...,...,...,...,...,...,...,...,...,...,...
595,IPCA,NaN,2023-01-02,2022,5.6676,5.6379,0.1180,5.5433,6.1636,40,1
596,IPCA,NaN,2023-01-02,2023,5.5072,5.4631,0.4784,4.5197,6.9300,40,1
597,IPCA,NaN,2023-01-02,2024,3.8155,3.8448,0.4013,3.0000,5.0000,36,1
598,IPCA,NaN,2023-01-02,2025,3.5731,3.5000,0.5140,3.0000,5.0000,32,1


In [4]:
dados_sgs.assign(variavel = "IDP")

,valor,variavel
Date,,
2021-01-01,38830.1,IDP
2021-02-01,45111.8,IDP
2021-03-01,46666.4,IDP
2021-04-01,50173.3,IDP
2021-05-01,48949.6,IDP
2021-06-01,43533.0,IDP
2021-07-01,46549.1,IDP
2021-08-01,50691.8,IDP
2021-09-01,50263.7,IDP


Seguindo a mesma lógica, podemos criar quantas colunas forem necessárias, com novos valores ou valores modificados de colunas preexistentes. Repare que você pode usar funções lambda que fazem rerefência às colunas e retornam uma Series de valores:

In [5]:
# Criando N colunas
dados_sgs.assign(
  variavel   = "IDP",                           # com escalar
  idp_bilhao = dados_sgs["valor"] / 1000,       # atribuindo uma pandas Series
  idp_log    = lambda x: np.log(x.idp_bilhao),  # com função lambda que retorna pandas Series
  idp_lag1   = lambda x: x.valor.shift(0),
  idp_lag2   = lambda x: x.valor.shift(0),
  ).tail(3)


,valor,variavel,idp_bilhao,idp_log,idp_lag1,idp_lag2
Date,,,,,,
2024-08-01,63901.3,IDP,63.9013,4.157340,63901.3,63901.3
2024-09-01,63378.8,IDP,63.3788,4.149129,63378.8,63378.8
2024-10-01,66025.5,IDP,66.0255,4.190041,66025.5,66025.5


In [6]:
# Alterando colunas
#Além de criar novas colunas, você pode alterar os valores de uma coluna preexistente:
dados_sgs.assign(
  valor_copia     = dados_sgs["valor"],
  valor           = lambda x: np.log(x.valor), # coluna original alterada
  valor_revertido = lambda x: np.exp(x.valor)
  ).tail(3)

,valor,valor_copia,valor_revertido
Date,,,
2024-08-01,11.065095,63901.3,63901.3
2024-09-01,11.056885,63378.8,63378.8
2024-10-01,11.097796,66025.5,66025.5


Para alterar diversas colunas com uma mesma operação 
(por exemplo, arredondar todas as colunas numéricas)
 e mantendo as demais colunas da tabela inalteradas, 
 é possível utilizar diversas abordagens de códigos.
  Talvez a forma mais simples seja utilizar a função 
  pipe() em conjunto com assign():



In [7]:
# Lista com nomes das colunas que quero arrendondar
colunas = ["Media", "Mediana", "DesvioPadrao", "Minimo", "Maximo"]
# Aplica np.round() sobre colunas do DataFrame
dados_focus.pipe(
  lambda y: y.assign(**y[colunas].applymap(np.round))
  )

/tmp/ipykernel_96463/1945371112.py:5: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  lambda y: y.assign(**y[colunas].applymap(np.round))


,Indicador,IndicadorDetalhe,Data,DataReferencia,Media,Mediana,DesvioPadrao,Minimo,Maximo,numeroRespondentes,baseCalculo
0,IGP-M,NaN,2023-02-10,2023,5.0,5.0,1.0,3.0,8.0,73,0
1,IGP-M,NaN,2023-02-10,2024,4.0,4.0,1.0,3.0,10.0,56,0
2,IGP-M,NaN,2023-02-10,2025,4.0,4.0,1.0,3.0,6.0,50,0
3,IGP-M,NaN,2023-02-10,2026,4.0,4.0,1.0,3.0,6.0,47,0
4,IGP-M,NaN,2023-02-10,2027,4.0,4.0,1.0,3.0,6.0,42,0
...,...,...,...,...,...,...,...,...,...,...,...
595,IPCA,NaN,2023-01-02,2022,6.0,6.0,0.0,6.0,6.0,40,1
596,IPCA,NaN,2023-01-02,2023,6.0,5.0,0.0,5.0,7.0,40,1
597,IPCA,NaN,2023-01-02,2024,4.0,4.0,0.0,3.0,5.0,36,1
598,IPCA,NaN,2023-01-02,2025,4.0,4.0,1.0,3.0,5.0,32,1


In [48]:
import numpy as np
dados_sgs.assign(valor_2=dados_sgs.valor*2,
                 valor_3=dados_sgs.valor**2,
                 valor_np_raiz=np.sqrt(dados_sgs.valor),
                 valor_log= lambda x: np.log(x.valor_3),
                 valor_exp=lambda x: np.exp(x.valor/1000)
                                     
                )

,valor,valor_2,valor_3,valor_np_raiz,valor_log,valor_exp
Date,,,,,,
2021-01-01,38830.1,77660.2,1.507777e+09,197.053546,21.133902,7.306311e+16
2021-02-01,45111.8,90223.6,2.035074e+09,212.395386,21.433798,3.906662e+19
2021-03-01,46666.4,93332.8,2.177753e+09,216.024073,21.501559,1.849098e+20
2021-04-01,50173.3,100346.6,2.517360e+09,223.993973,21.646477,6.165770e+21
2021-05-01,48949.6,97899.2,2.396063e+09,221.245565,21.597093,1.813599e+21
2021-06-01,43533.0,87066.0,1.895122e+09,208.645633,21.362549,8.056412e+18
2021-07-01,46549.1,93098.2,2.166819e+09,215.752404,21.496526,1.644437e+20
2021-08-01,50691.8,101383.6,2.569659e+09,225.148396,21.667039,1.035545e+22
2021-09-01,50263.7,100527.4,2.526440e+09,224.195673,21.650077,6.749126e+21


Para alterar diversas colunas com uma mesma operação (por exemplo, 
arredondar todas as colunas numéricas) e mantendo as demais colunas da tabela inalteradas,
 é possível utilizar diversas abordagens de códigos. Talvez a forma mais simples seja utilizar a função pipe() em conjunto com assign():



In [26]:
# Lista com nomes das colunas que quero arrendondar
colunas = ["Media", "Mediana", "DesvioPadrao", "Minimo", "Maximo"]
# Aplica np.round() sobre colunas do DataFrame
dados_focus.pipe(
  lambda y: y.assign(**y[colunas].map(np.round))
  )

,Indicador,IndicadorDetalhe,Data,DataReferencia,Media,Mediana,DesvioPadrao,Minimo,Maximo,numeroRespondentes,baseCalculo
0,IGP-M,NaN,2023-02-10,2023,5.0,5.0,1.0,3.0,8.0,73,0
1,IGP-M,NaN,2023-02-10,2024,4.0,4.0,1.0,3.0,10.0,56,0
2,IGP-M,NaN,2023-02-10,2025,4.0,4.0,1.0,3.0,6.0,50,0
3,IGP-M,NaN,2023-02-10,2026,4.0,4.0,1.0,3.0,6.0,47,0
4,IGP-M,NaN,2023-02-10,2027,4.0,4.0,1.0,3.0,6.0,42,0
...,...,...,...,...,...,...,...,...,...,...,...
595,IPCA,NaN,2023-01-02,2022,6.0,6.0,0.0,6.0,6.0,40,1
596,IPCA,NaN,2023-01-02,2023,6.0,5.0,0.0,5.0,7.0,40,1
597,IPCA,NaN,2023-01-02,2024,4.0,4.0,0.0,3.0,5.0,36,1
598,IPCA,NaN,2023-01-02,2025,4.0,4.0,1.0,3.0,5.0,32,1


Exemplo de Usar a Função Lambda

In [ ]:
#Funções que precisam do **
#Essas funções aceitam argumentos nomeados (keyword arguments). 
# O ** desempacota um dicionário para fornecer esses argumentos.


# Criando um dicionário dinâmico para várias colunas
df = pd.DataFrame({'A': [1, 2], 'B': [3, 4]})

# Criar um dicionário de novas colunas
novas_colunas = {'C': df['A'] + df['B'], 'D': df['B'] * 2,'E':np.exp(df['A']),'F':np.log(df['B'])}
df=df.assign(**novas_colunas)
df

,A,B,C,D,E,F
0,1,3,4,6,2.718282,1.098612
1,2,4,6,8,7.389056,1.386294


In [22]:
#Exemplo 2: str.format() com placeholders dinâmicos
valores = {'nome': 'Maria', 'idade': 25}

# Usando ** para passar os valores
mensagem = "Olá, meu nome é {nome} e tenho {idade} anos.".format(**valores)
#mensagem=f'Olá, meu nome é {nome} e tenho {idade} anos.'    ''''It will go wrong'''
print(mensagem)


Olá, meu nome é Maria e tenho 25 anos.


Essas funções geralmente trabalham diretamente com objetos como listas, DataFrames ou Series e não exigem argumentos nomeados.

Exemplo 1: Aplicar uma função diretamente em colunas do DataFrame

In [25]:
import numpy as np

# DataFrame exemplo
df = pd.DataFrame({'A': [1.5, 2.7], 'B': [3.1, 4.8]})

# Aplicando np.round diretamente
df[['A', 'B']] = df[['A', 'B']].map(np.round)
df


,A,B
0,2.0,3.0
1,3.0,5.0


Resumo
Precisa de **: Quando a função espera argumentos nomeados.
Exemplos: .assign(), str.format(), funções que explicitamente usam **kwargs.
Não precisa de **: Quando a função lida diretamente com objetos como listas, Series ou DataFrames.
Exemplos: .applymap(), .apply(), criação direta de objetos como DataFrame.





